In [1]:
import torch
import random
import pickle
from get_tensor import get_samples_per_sky
from cnn_models.model_templates import SkylightData, TrainingHelper
from cnn_models.sixth_gen import SixthGen

In [2]:
skies_for_demo = {  # first 4 clear, next 4 scattered, last 3 overcast
    "5/26/2013 15:15",
    "5/27/2013 10:15",
    "7/26/2013 13:15",
    "9/24/2013 15:39",
    "5/12/2013 13:00",
    "5/26/2013 12:30",
    "7/29/2013 10:00",
    "9/26/2013 11:50",
    "4/14/2013 11:36",
    "4/15/2013 08:40",
    "7/29/2013 13:30",
}

scattered_skies_data = SkylightData(
    "other_data/coords-scattered-hdr.csv", get_samples_per_sky
)


load = True
if load:
    with open("other_data/development_data_save", "rb") as f:
        development_data = pickle.load(f)
else:
    with open("other_data/development_data_save", "wb") as f:
        development_data = scattered_skies_data.create_development_data(
            training_ratio=0.75, excluded_skies=skies_for_demo
        )
        pickle.dump(development_data, f)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SixthGen()
model.to(device)

lr = 0.000001
helper = TrainingHelper(
    data=development_data,
    model=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=lr),
    criterion=torch.nn.MSELoss(reduction="sum"),
    device=device,
)

In [ ]:
report = helper.training_loop(
    number_of_epochs=200, batch_size=420, model_used="SixthGen", lr=lr
)

0	Training Loss: 64412.71		Testing Loss: 9599.69
	               +0.00%	                      +0.00%
Minutes Training: 0



In [ ]:
model.to(torch.device("cpu"))

pred = model(development_data.test_data.x_image_branch, development_data.test_data.x_feature_branch).detach()

truth = development_data.test_data.y

In [ ]:
diff_squared = ((pred - truth) * development_data.data_aggregates.data_range) ** 2
diff_squared.sum(dim=1).mean().sqrt() * 100

In [ ]:
report.save('other_data')